In [1]:
''' identify TC environments (time and space - depending on R34?) from usa source in IBTrACS '''
import numpy as np
import netCDF4 as nc
import matplotlib
import matplotlib.pyplot as plt
import datetime
import os
import sys

In [2]:
''' LOAD TC tracks data '''
path_tracks = '/Users/chi/Documents/Data/IBTrACS/'
ds_tracks = nc.Dataset(path_tracks+'IBTrACS.ALL.v04r01.nc')
vars_str = ['time','usa_lon','usa_lat','basin','name','usa_status','usa_sshs','usa_pres','usa_wind','usa_r34','usa_r64','storm_speed','storm_dir']
# time:units = "days since 1858-11-17 00:00:00" ;
for i, var_str in enumerate( vars_str ):
    str_exec = var_str + '= ds_tracks.variables["'+ var_str +'"][:]'
    exec(str_exec)
    print(i, str_exec, eval(var_str).shape)
''' convert time to timestamp '''
timestamp_track = np.nan*np.ones( time.shape )#).tolist()
for i in range( time.shape[0] ):
    for j in range( time.shape[1] ):
        if isinstance( time[i,j],np.float64 ):
            timestamp_track[i][j] = (datetime.datetime(1858,11,17)+datetime.timedelta(days=time[i][j])).timestamp()
# plt.plot(time[1,:])
''' decode basin & name'''
name = np.char.decode(name)
basin = np.char.decode(basin)
usa_status = np.char.decode(usa_status)

0 time= ds_tracks.variables["time"][:] (13414, 360)
1 usa_lon= ds_tracks.variables["usa_lon"][:] (13414, 360)
2 usa_lat= ds_tracks.variables["usa_lat"][:] (13414, 360)
3 basin= ds_tracks.variables["basin"][:] (13414, 360, 2)
4 name= ds_tracks.variables["name"][:] (13414, 128)
5 usa_status= ds_tracks.variables["usa_status"][:] (13414, 360, 2)
6 usa_sshs= ds_tracks.variables["usa_sshs"][:] (13414, 360)
7 usa_pres= ds_tracks.variables["usa_pres"][:] (13414, 360)
8 usa_wind= ds_tracks.variables["usa_wind"][:] (13414, 360)
9 usa_r34= ds_tracks.variables["usa_r34"][:] (13414, 360, 4)
10 usa_r64= ds_tracks.variables["usa_r64"][:] (13414, 360, 4)
11 storm_speed= ds_tracks.variables["storm_speed"][:] (13414, 360)
12 storm_dir= ds_tracks.variables["storm_dir"][:] (13414, 360)


In [4]:
''' choose years for slicing tracks'''
year = ['2021','2022','2023','2024']

In [5]:
''' Slice tracks in specified year(s) & the circle of R34 & Add HU Name '''
vars_str_want = ['name','time','usa_lon','usa_lat','usa_sshs','usa_pres','usa_wind','usa_r34','usa_r64','storm_speed','storm_dir']
for var_str in vars_str_want:
    str_exec = var_str + '_want = []'
    exec(str_exec)
''' go through timestamp to find data in years wanted '''
for i in range( timestamp_track.shape[0] ):
    year_this_track = str( datetime.datetime.fromtimestamp(timestamp_track[i,0]).year )
    if (year_this_track in year) & (basin[i,0,0] == 'N') & (basin[i,0,1] == 'A'):
        ''' name '''
        test = (name[i,:]).tolist()
        test = ''.join(test)
        name_want.append( test )
        # print(name_want)
        # ''' usa_status '''
        # for j in range( timestamp_track.shape[1] ):
        #     test = (usa_status[i,j,:]).tolist()
        ''' append tracks to list of variables '''
        edge_ok = np.ma.flatnotmasked_edges(usa_lon[i,:])
        for v, var_str in enumerate( vars_str_want ):
            if var_str == 'time':
                time_want.append( timestamp_track[i,edge_ok[0]:edge_ok[1]+1] )
            elif var_str == 'name':
                pass
            elif  ('r34' in var_str) | ('r64' in var_str):
                str_exec = var_str+'_want.append('+var_str+'[i,edge_ok[0]:edge_ok[1]+1,:] )'
                exec(str_exec)
            else:
                str_exec = var_str+'_want.append('+var_str+'[i,edge_ok[0]:edge_ok[1]+1] )'
                exec(str_exec)
                
''' put list to array '''
for var_str in vars_str_want:
    print(var_str)
    print( len(eval(var_str+'_want')) )

name
79
time
79
usa_lon
79
usa_lat
79
usa_sshs
79
usa_pres
79
usa_wind
79
usa_r34
79
usa_r64
79
storm_speed
79
storm_dir
79


In [6]:
''' stack the "want" variables to a np.array '''
# vars_str_want = ['time','usa_lon','usa_lat','usa_sshs','usa_wind','usa_r34']
vars_str_want = ['time','usa_lon','usa_lat','usa_sshs','usa_pres','usa_wind','usa_r34','usa_r64','storm_speed','storm_dir']
units_str_want = ['(Name of system)','(timestamp)','(degE)','deg(N)','()','(mb)','(kts)','(nautical miles)','(nautical miles)','(kts)','(degrees)']
for i in range( len(name_want) ):
    print(name_want[i])
    ntime_this_system = len(time_want[i])
    ''' initiate an array with column length = lenght of this system '''
    array_this_system = np.array( [name_want[i]]*ntime_this_system )
    for v, var_str in enumerate( vars_str_want ):
        # print(var_str)
        if ('r34' in var_str) | ('r64' in var_str):
            test = eval(var_str+'_want[i][:][:]')
        else:
            test = eval(var_str+'_want[i][:]')
        array_this_system = np.column_stack( (array_this_system, test) )
    # print(array_this_system.shape)
    if i == 0:
        array_out = array_this_system    
    else:
        array_out = np.vstack( (array_out, array_this_system) )
    # print(array_out.shape)

''' write np array to text file '''
fn_out = 'timeseries_IBTrACS_NA_'+'-'.join(year)+'.txt'
vars_str_want.insert(0,'name')
header_out = ', '.join( vars_str_want )+'\n'+', '.join( units_str_want )
print(header_out)
np.savetxt(fn_out, array_out, fmt='%s', delimiter=' ',header=header_out)

ANA
BILL
CLAUDETTE
DANNY
ELSA
FRED
GRACE
HENRI
IDA
KATE
JULIAN
LARRY
MINDY
NICHOLAS
ODETTE
ROSE
PETER
SAM
TERESA
VICTOR
WANDA
ALEX
BONNIE
COLIN
UNNAMED
DANIELLE
EARL
FIONA
GASTON
IAN
HERMINE
UNNAMED
UNNAMED
JULIA
KARL
LISA
MARTIN
NICOLE
UNNAMED
ARLENE
BRET
CINDY
DON
GERT
EMILY
FRANKLIN
HAROLD
IDALIA
JOSE
KATIA
LEE
MARGOT
NIGEL
OPHELIA
PHILIPPE
RINA
SEAN
TAMMY
UNNAMED
UNNAMED
ALBERTO
CHRIS
BERYL
DEBBY
ERNESTO
FRANCINE
GORDON
UNNAMED
HELENE
JOYCE
ISAAC
KIRK
LESLIE
MILTON
OSCAR
KRISTY:NADINE
PATTY
RAFAEL
SARA
name, time, usa_lon, usa_lat, usa_sshs, usa_pres, usa_wind, usa_r34, usa_r64, storm_speed, storm_dir
(Name of system), (timestamp), (degE), deg(N), (), (mb), (kts), (nautical miles), (nautical miles), (kts), (degrees)
